### Run this code after running the SAS code

In [1]:
# Import necessary modules
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

# INPUT / OUTPUT PATHS
INPUT_CSV = Path("trauma_winners_by_zip.csv")
GEOJSON_FILE = Path("ga_zipcodes.geojson")

OUT_DRIVE_MAP = Path("trauma_driving_map.html")
OUT_CONNECTIONS = Path("trauma_connections.html")

# LOAD & VALIDATE DATA
if not INPUT_CSV.exists():
    raise FileNotFoundError(f"Missing: {INPUT_CSV.resolve()}")
if not GEOJSON_FILE.exists():
    raise FileNotFoundError(f"Missing: {GEOJSON_FILE.resolve()}")

df = pd.read_csv(INPUT_CSV)

expected_cols = {
    "zip", "CITY", "COUNTY", "hosp_name",
    "zip_lon", "zip_lat", "hosp_lon", "hosp_lat",
    "miles_geo", "miles_drive", "time_sec", "time_text", "metric_type"
}
missing = expected_cols - set(df.columns)
if missing:
    print("Warning: Missing expected columns:", sorted(missing))

# Drop rows missing coordinates
df = df.dropna(subset=["zip_lon", "zip_lat", "hosp_lon", "hosp_lat"]).copy()
df["zip"] = df["zip"].astype(str).str.zfill(5)

# Ensure numeric and non-missing time_min
df["time_sec"] = pd.to_numeric(df["time_sec"], errors="coerce")
df["time_min"] = df["time_sec"] / 60
overall_median = df["time_min"].median()
df["time_min"] = df["time_min"].fillna(overall_median)

# Keep only driving metrics
df_drive = df[df["metric_type"].str.contains("Driving", case=False, na=False)].copy()

# LOAD GEOJSON
with open(GEOJSON_FILE) as f:
    ga_zipcodes = json.load(f)

# DRIVING DISTANCE & TIME MAP
if df_drive.empty:
    print("No driving records found.")
else:
    df_drive["hover_text"] = (
        "<b>ZIP:</b> " + df_drive["zip"].astype(str) +
        "<br><b>City:</b> " + df_drive["CITY"].astype(str) +
        "<br><b>County:</b> " + df_drive["COUNTY"].astype(str) +
        "<br><b>Hospital:</b> " + df_drive["hosp_name"].astype(str) +
        "<br><b>Driving Distance (mi):</b> " + df_drive["miles_drive"].round(1).astype(str) +
        "<br><b>Driving Time:</b> " + df_drive["time_text"].astype(str)
    )

    fig_drive = px.choropleth_map(
        df_drive,
        geojson=ga_zipcodes,
        locations="zip",
        featureidkey="properties.ZCTA5CE10",
        color="miles_drive",
        color_continuous_scale="Viridis",
        custom_data=["hover_text"],
        title="Driving Distance & Time to Nearest Trauma Center (GA)",
        center={"lat": 32.9, "lon": -83.3},
        zoom=6
    )
    fig_drive.update_traces(
        hovertemplate="%{customdata[0]}<extra></extra>",
        marker_line_width=0.5,
        marker_line_color="white"
    )
    fig_drive.update_geos(fitbounds="locations", visible=False)
    fig_drive.write_html(OUT_DRIVE_MAP, include_plotlyjs="cdn")
    print(f"Wrote {OUT_DRIVE_MAP.resolve()}")

# CONNECTION MAP (ZIP → Hospital links with straight-line distance)
if df_drive.empty:
    print("No driving data available for connection map.")
else:
    fig_conn = go.Figure()

    # combine all lines into one trace (clean legend)
    all_lines_lon, all_lines_lat = [], []
    for row in df_drive.itertuples(index=False):
        all_lines_lon += [row.zip_lon, row.hosp_lon, None]
        all_lines_lat += [row.zip_lat, row.hosp_lat, None]

    fig_conn.add_trace(go.Scattergeo(
        lon=all_lines_lon,
        lat=all_lines_lat,
        mode="lines",
        line=dict(width=1, color="rgba(0,90,200,0.25)"),
        hoverinfo="skip",
        showlegend=False
    ))

    # ZIP centroids (blue dots) with hover showing straight-line distance
    fig_conn.add_trace(go.Scattergeo(
        lon=df_drive["zip_lon"],
        lat=df_drive["zip_lat"],
        mode="markers",
        marker=dict(size=3, color="navy"),
        name="ZIP centroids",
        hovertext=(
            "<b>ZIP:</b> " + df_drive["zip"].astype(str) +
            "<br><b>City:</b> " + df_drive["CITY"].astype(str) +
            "<br><b>Hospital:</b> " + df_drive["hosp_name"].astype(str) +
            "<br><b>Straight-line Distance:</b> " + df_drive["miles_geo"].round(1).astype(str) + " mi"
        ),
        hoverinfo="text"
    ))

    # Trauma centers (red diamonds)
    fig_conn.add_trace(go.Scattergeo(
        lon=df_drive["hosp_lon"],
        lat=df_drive["hosp_lat"],
        mode="markers",
        marker=dict(size=8, color="crimson", symbol="diamond"),
        name="Trauma centers",
        hovertext=df_drive["hosp_name"],
        hoverinfo="text"
    ))

    fig_conn.update_geos(scope="usa", projection_type="albers usa", showland=True)
    fig_conn.update_layout(
        title="Straight-Line Connections: ZIP → Nearest Trauma Center (GA)",
        legend=dict(
            title="Legend",
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5
        ),
        margin=dict(l=0, r=0, t=60, b=0)
    )
    fig_conn.write_html(OUT_CONNECTIONS, include_plotlyjs="cdn")
    print(f"Wrote {OUT_CONNECTIONS.resolve()}")

print("\nDriving map and connection map successfully created (hover includes straight-line distance).")

Wrote G:\My Drive\GitHub\plots\codes\trauma_driving_map.html
Wrote G:\My Drive\GitHub\plots\codes\trauma_connections.html

Driving map and connection map successfully created (hover includes straight-line distance).
